<a href="https://colab.research.google.com/github/puroong/2025-SSUML/blob/main/Term_Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
# print(IN_COLAB)

if IN_COLAB:
    !git clone https://github.com/ssuai/project_finance.git
    !git clone https://github.com/ssuai/ISLP.git
    #!pip install ISLP # This takes too much time

Cloning into 'project_finance'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 40 (delta 16), reused 5 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 40.84 KiB | 4.54 MiB/s, done.
Resolving deltas: 100% (16/16), done.
Cloning into 'ISLP'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 63 (delta 14), reused 53 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (63/63), 2.34 MiB | 12.02 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [ ]:
from ISLP.utils import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize)
import statsmodels.api as sm
from ISLP.models import sklearn_sm
from sklearn.model_selection import \
     (cross_validate,
      KFold,
      ShuffleSplit)

import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
import pandas as pd
from datetime import datetime

# helper function for reading datatset
def read_data(file_path):
    df = pd.read_csv(file_path)
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d') # convert it to datatime

    return df

df_train = read_data('project_finance/data/kospi_train.csv')
df_test = read_data('project_finance/data/kospi_test.csv')

len(df_train), len(df_test)

(986, 244)

In [33]:
import itertools

def column_combination():
  significant_columns = ['Open', 'Low', 'High', 'Close']

  for r in range(1, len(significant_columns)+1):
      l = itertools.combinations(significant_columns, r)
      for i in l:
          yield list(i)

result_dict = {}

X_test = None

for terms in column_combination():
#terms = df_train.columns.drop('Close').drop('Date').drop('Volume')
  ms = MS(terms)
  X_test = MS(terms).fit_transform(df_test)[:-1].reset_index(drop=True)
  y_test = df_test['Close'][1:].reset_index(drop=True)
  #X_test = df_test[terms][:-1].reset_index(drop=True)

  X = MS(terms).fit_transform(df_train)[:-1].reset_index(drop=True)
  y = df_train['Close'][1:].reset_index(drop=True)

  model = sm.OLS(y, X)
  result = model.fit()
  """
  hp_model = sklearn_sm(sm.OLS,
                      MS(terms))

  cv_results = cross_validate(hp_model,
                              X,
                              y,
                              cv=df_train.shape[0]-1)
  cv_err = np.mean(cv_results['test_score'])
  result_dict[str(terms)] = cv_err
  """

  y_pred = result.predict(X_test)
  mse = mean_squared_error(y_test, y_pred)

  result_dict[str(terms)] = mse

result_dict

/content/ISLP/models/columns.py:151: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if is_categorical[i]:
/content/ISLP/models/columns.py:151: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if is_categorical[i]:
/content/ISLP/models/columns.py:151: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if is_categorical[i]:
/content/ISLP/models/columns.py:151: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a fut

{"['Open']": 823.6774440646197,
 "['Low']": 663.9948428076613,
 "['High']": 679.1228487979695,
 "['Close']": 577.4147229710437,
 "['Open', 'Low']": 664.1932105183278,
 "['Open', 'High']": 683.450441387073,
 "['Open', 'Close']": 575.0542798521665,
 "['Low', 'High']": 631.54661743017,
 "['Low', 'Close']": 578.1347704740831,
 "['High', 'Close']": 576.6832425015116,
 "['Open', 'Low', 'High']": 610.6224268177685,
 "['Open', 'Low', 'Close']": 579.1576847598936,
 "['Open', 'High', 'Close']": 583.3395455288195,
 "['Low', 'High', 'Close']": 581.3119473036851,
 "['Open', 'Low', 'High', 'Close']": 582.6209328666364}

In [ ]:
kospi_test = load_data("kospi_test")
terms = kospi_test.columns.drop('Close').drop('Date').drop('Volume')

new_X = MS(terms).fit_transform(kospi)[:-prev_days].reset_index(drop=True)

new_predictions = result.get_prediction(new_X);
new_predictions.predicted_mean

FileNotFoundError: [Errno 2] No such file or directory: '/content/ISLP/data/kospi_test.csv'